# Certified Finetuning of a Classifier on the OCT-MNIST Dataset

In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import tqdm
import abstract_gradient_training as agt
from abstract_gradient_training import AGTConfig
from abstract_gradient_training import model_utils
from models.deepmind import DeepMindSmall 
from datasets import oct_mnist

## Pre-train the model

Exclude class 2 (Drusen) from the pretraining.

In [2]:
# set up pre-training
torch.manual_seed(1)
device = torch.device("cuda:0")
pretrain_batchsize = 100
pretrain_n_epochs = 20
pretrain_learning_rate = 0.001

In [3]:
# define model, dataset and optimizer
model = DeepMindSmall(1, 1)
dl_pretrain, _ = oct_mnist.get_dataloaders(pretrain_batchsize, exclude_classes=[2], balanced=True)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_learning_rate)
model = model.to(device)

In [4]:
if os.path.exists(".models/medmnist.ckpt"):
    model.load_state_dict(torch.load(".models/medmnist.ckpt"))
else:  # pre-train the model
    progress_bar = tqdm.trange(pretrain_n_epochs, desc="Epoch", )
    for epoch in progress_bar:
        for i, (x, u) in enumerate(dl_pretrain):
            # Forward pass
            u, x = u.to(device), x.to(device)
            output = model(x)
            loss = criterion(output.squeeze().float(), u.squeeze().float())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                progress_bar.set_postfix(loss=loss.item())
    # save the model
    with open(".models/medmnist.ckpt", "wb") as file:
        torch.save(model.state_dict(), file)

/tmp/ipykernel_3892008/122059171.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(".models/medmnist.ckpt"))


## Fine-tune the model on the private Drusen data

In [5]:
# set up fine-tuning parameters
batchsize = 3000
config = AGTConfig(
    fragsize=1000,
    learning_rate=0.1,
    n_epochs=2,
    k_unlearn=50,
    forward_bound="interval",
    device="cuda:0",
    backward_bound="interval",
    loss="binary_cross_entropy",
    lr_decay=0.3,
    lr_min=0.001,
    log_level="DEBUG",
)

In [6]:
# get dataloaders, train dataloader is a mix of drusen and the "healthy" class
dl_train, _ = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1], balanced=True)
_, dl_test_drusen = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1, 3])
_, dl_test_other = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[2])
_, dl_test_all = oct_mnist.get_dataloaders(batchsize, 1000)

In [7]:
# evaluate the pre-trained model
conv_layers = model[0:5]
linear_layers = model[5:-1]
conv_transform = model_utils.get_conv_model_transform(conv_layers)
param_l, param_n, param_u = model_utils.get_parameters(linear_layers)

drusen_acc = agt.test_metrics.test_accuracy(
    param_l, param_n, param_u, *next(iter(dl_test_drusen)), transform=conv_transform
)
other_acc = agt.test_metrics.test_accuracy(
    param_l, param_n, param_u, *next(iter(dl_test_other)), transform=conv_transform
)
all_acc = agt.test_metrics.test_accuracy(param_l, param_n, param_u, *next(iter(dl_test_all)), transform=conv_transform)

print("=========== Pre-trained model accuracy ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {other_acc[1]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}")

=========== Pre-trained model accuracy ===========
Class 2 (Drusen) : nominal = 0.46
Classes 0, 1, 3  : nominal = 0.96
All Classes      : nominal = 0.84


In [8]:
# fine-tune the model using abstract gradient training (keeping the convolutional layers fixed)
param_l, param_n, param_u = agt.unlearning_certified_training(
    linear_layers, config, dl_train, dl_test_drusen, transform=conv_transform
)

[AGT] [INFO    ] [17:00:47] =================== Starting Unlearning Certified Training ===================
[AGT] [DEBUG   ] [17:00:47] 	Optimizer params: n_epochs=2, learning_rate=0.1, l1_reg=0.0, l2_reg=0.0
[AGT] [DEBUG   ] [17:00:47] 	Learning rate schedule: lr_decay=0.3, lr_min=0.001, early_stopping=True
[AGT] [DEBUG   ] [17:00:47] 	Unlearning parameter: k_unlearn=50
[AGT] [DEBUG   ] [17:00:47] 	Clipping: gamma=inf, method=clamp
[AGT] [DEBUG   ] [17:00:47] 	Noise: type=gaussian, sigma=0
[AGT] [DEBUG   ] [17:00:47] 	Bounding methods: forward=interval, loss=binary_cross_entropy, backward=interval
[AGT] [INFO    ] [17:00:47] Starting epoch 1
[AGT] [DEBUG   ] [17:00:47] Initialising dataloader batchsize to 3000
[AGT] [INFO    ] [17:00:47] Training batch 1: Network eval bounds=(0.46, 0.46, 0.46), W0 Bound=0.0 
[AGT] [INFO    ] [17:00:47] Training batch 2: Network eval bounds=(0.72, 0.76, 0.76), W0 Bound=0.0748 
[AGT] [INFO    ] [17:00:48] Training batch 3: Network eval bounds=(0.76, 0.84

In [9]:
# evaluate the fine-tuned model
drusen_acc = agt.test_metrics.test_accuracy(param_l, param_n, param_u, *next(iter(dl_test_drusen)), transform=conv_transform)
other_acc = agt.test_metrics.test_accuracy(param_l, param_n, param_u, *next(iter(dl_test_other)), transform=conv_transform)
all_acc = agt.test_metrics.test_accuracy(param_l, param_n, param_u, *next(iter(dl_test_all)), transform=conv_transform)

print("=========== Fine-tuned model accuracy + bounds ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}, certified bound = {drusen_acc[0]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {other_acc[1]:.2g}, certified bound = {other_acc[0]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}, certified bound = {all_acc[0]:.2g}")

=========== Fine-tuned model accuracy + bounds ===========
Class 2 (Drusen) : nominal = 0.86, certified bound = 0.11
Classes 0, 1, 3  : nominal = 0.88, certified bound = 0.48
All Classes      : nominal = 0.88, certified bound = 0.39
